In [25]:
#if you haven't installed google-api-client for python: 
#!pip install --upgrade google-api-python-client --quiet

In [37]:
from googleapiclient.discovery import build
import pandas as pd
import numpy as np

In [1]:
#Use the api key created on your own 
api_key='XXXXXXXXXXXXX'

In [28]:
api_service_name = "youtube"
api_version = "v3"
youtube = build(
        api_service_name, api_version, developerKey=api_key)
Playlist_id="PLWKjhJtqVAbmoiNlqLJg1gxEjEuKHHcn_"
def video_ids(youtube,playlist_id):
    videoIDs=[]
    request = youtube.playlistItems().list(
        part="contentDetails",
        playlistId=Playlist_id,
        maxResults=50
    )
    response = request.execute()
    for items in response['items']:
        a=items['contentDetails']['videoId']
        videoIDs.append(a)
        nextPageToken=response.get('nextPageToken')
        while nextPageToken is not None:
            request = youtube.playlistItems().list(
                part="contentDetails",
                playlistId=Playlist_id,
                maxResults=50,
                pageToken=nextPageToken
            )
            response = request.execute()
            for items in response['items']:
                a=items['contentDetails']['videoId']
                videoIDs.append(a)
                nextPageToken=response.get('nextPageToken')

    return videoIDs

In [29]:
videos=video_ids(youtube,Playlist_id)
a=len(videos)
a

92

In [52]:
def get_video_data(youtube,videos):
        
    data_df=[]
    for i in range(0,a,50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(videos[i:i+50])
        )
        response = request.execute()
        for items in response['items']:
            data={'Video_ID':items['id'],
                  'Video_Name':items['snippet']['title'],
                  'Date':items['snippet']['publishedAt'],
                  'View_Count':items['statistics']['viewCount'],
                  'Like_Count':items['statistics']['likeCount'],
                  'Comment_Count':items['statistics']['commentCount'],
                  'Tags':items['snippet']['tags']
                 }
            data_df.append(data)
    index=range(1,len(videos)+1)
    video_df=pd.DataFrame(data_df,index=index)
    return video_df

In [54]:

videos_df=get_video_data(youtube,videos)
videos_df

,Video_ID,Video_Name,Date,View_Count,Like_Count,Comment_Count,Tags
1,t2CEgPsws3U,Data Structures and Algorithms in JavaScript -...,2018-09-04T18:16:17Z,996206,23169,506,"[data structures and algorithms, data structur..."
2,iLx4ma8ZqvQ,...spread operator and rest operator - Beau te...,2017-03-22T18:04:50Z,147948,5486,173,"[spread, rest, operator, arrays, condense elem..."
3,r6SnMjsLrBk,"Objects, part 2: Beau teaches JavaScript",2017-03-21T19:12:12Z,16351,255,5,"[tutorial, lookups, delete, remove, nested obj..."
4,r7v6EIiHfVA,Logical operators && TRICKS with short-circuit...,2017-03-20T16:44:04Z,67109,1040,38,"[javascript, short circuit, short-circuit, eva..."
5,bK7I79hcm08,Queues & Priority Queues - Beau teaches JavaSc...,2017-03-19T19:32:07Z,42117,544,27,"[queue, dequeue, enqueue, priority, data struc..."
...,...,...,...,...,...,...,...
88,5cU1ILGy6dM,Binary Search Tree - Beau teaches JavaScript,2017-03-26T19:50:46Z,110561,1797,78,"[tree, binary search tree, data structure, nod..."
89,QnLBGxtteV8,"Clean Code: Testing, Concurrency, & Error Hand...",2017-03-25T16:41:08Z,9295,211,1,"[clean code, async, await, promises, testing, ..."
90,v9zgD8wjtbw,While / Do While - Beau teaches JavaScript,2017-03-24T16:20:56Z,33588,628,19,"[while, do while, do, loops, basics, javascript]"
91,Vd_Z1bYGrCM,String Basics - Beau teaches JavaScript,2017-03-23T17:15:31Z,29128,365,21,"[strings, string literals, basics, training]"


In [55]:
videos_df.to_csv("FreeCodeCamp-Playlist-data.csv",index=False)

In [56]:
df=pd.read_csv("FreeCodeCamp-Playlist-data.csv")
df.head()

,Video_ID,Video_Name,Date,View_Count,Like_Count,Comment_Count,Tags
0,t2CEgPsws3U,Data Structures and Algorithms in JavaScript -...,2018-09-04T18:16:17Z,996206,23169,506,"['data structures and algorithms', 'data struc..."
1,iLx4ma8ZqvQ,...spread operator and rest operator - Beau te...,2017-03-22T18:04:50Z,147948,5486,173,"['spread', 'rest', 'operator', 'arrays', 'cond..."
2,r6SnMjsLrBk,"Objects, part 2: Beau teaches JavaScript",2017-03-21T19:12:12Z,16351,255,5,"['tutorial', 'lookups', 'delete', 'remove', 'n..."
3,r7v6EIiHfVA,Logical operators && TRICKS with short-circuit...,2017-03-20T16:44:04Z,67109,1040,38,"['javascript', 'short circuit', 'short-circuit..."
4,bK7I79hcm08,Queues & Priority Queues - Beau teaches JavaSc...,2017-03-19T19:32:07Z,42117,544,27,"['queue', 'dequeue', 'enqueue', 'priority', 'd..."
